In [24]:
import datetime
import io
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from alerce.core import Alerce
# Getting classes for a classifier and version


In [25]:
r = requests.post(
    "https://api.fink-portal.org/api/v1/latests",
    json={
        "class": "Microlensing candidate",
        "output-format": "json",
        "output-format": "json",
        "n": 2000,
    },
)
pdf = pd.read_json(io.BytesIO(r.content))
#pdf.to_csv("test_query_2k_microlensing_Aug2025.csv")

In [35]:
#from astroquery.ipac.ned import Ned
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from astropy import units as u

#Ned.clear_cache()
#result_table = Ned.query_region(coords, radius=1/60. * u.deg)
coords = SkyCoord(ra=pdf["i:ra"].array*u.deg, dec=pdf["i:dec"].array*u.deg, frame='icrs')
custom_simbad = Simbad()
custom_simbad.add_votable_fields('otype')
result = custom_simbad.query_region(coords, radius=1/60. * u.deg)
mask_galaxy = result['otype'] == 'G'
galaxies = result[mask_galaxy]


In [4]:
year = str(datetime.datetime.now().year)[2:]
last_year = str(datetime.datetime.now().year - 1)[2:]
unique_recent = np.concatenate(
    [
        pdf[pdf["i:objectId"].str.contains("ZTF{}".format(year))][
            "i:objectId"
        ].unique(),
        pdf[pdf["i:objectId"].str.contains("ZTF{}".format(last_year))][
            "i:objectId"
        ].unique(),
    ]
)

In [8]:
object_names = [str(x) for x in unique_recent] # object_names

In [6]:
#Download all lightcurves and classifications fink
#for object_id in object_names:
#    r = requests.post(
#        "https://api.fink-portal.org/api/v1/objects",
#        json={"objectId": object_id, "output-format": "json"},
#    )   
#    pdf = pd.read_json(io.BytesIO(r.content))
#    filename = f"{object_id}.csv"
#    time.sleep(1)
#    pdf.to_csv(filename, index=False)

In [ ]:
#Download all classifications ALeRCE
#import os
#from alerce.core import Alerce
#alerce = Alerce()
#for object_id in object_names:
#    print(object_id)
#    filename = f"ALeRCE_classifiers_{object_id}.csv"
#    if not os.path.exists(filename):
#        try:
#            probabilities = alerce.query_probabilities(object_id)
#            prob_pd = pd.DataFrame.from_dict(probabilities)
#            prob_pd.to_csv(filename, index=False)
#        except Exception as e:
#            print("Unavailable {}".format(e))


In [ ]:
def tau_500(l,b,silence=False):
    """
    Return the closest value for a given latitude (b)
    #MROZ b bins, ignoring longitude limits
    #https://iopscience.iop.org/0067-0049/249/1/16/suppdata/apjsab9366t2_ascii.txt?doi=10.3847/1538-4365/ab9366
    Parameters:
        l (float): longitude
        b (float): latitude
        silence (bool): longitude out of range message silenced

    Returns:
        tau (float): The corresponding tau value. np.nan outside
    """
    if not (240 < l < 330):
        if silence == False:
            print("Warning: Longitude out of range")
    if -7 < b < -5:
        return 0.076
    elif -5 < b < -3:
        return 0.131
    elif -3 < b < -1:
        return 0.277
    elif -1 < b < 1:
        return 0.119
    elif 1 < b < 3:
        return 0.094
    elif 3 < b < 5:
        return 0.155
    elif 5 < b < 7:
        return 0.08
    else:
        return np.nan

tau_500_vec = np.vectorize(tau_500)



In [ ]:
from astropy.coordinates import Galactic, SkyCoord
from astropy import units as u
c = SkyCoord(ra=np.array(pdf["i:ra"].array)*u.degree, 
             dec=np.array(pdf["i:dec"].array)*u.degree, frame='icrs')

tau_values = tau_500(c.galactic.l.deg,c.galactic.b.deg)
plt.scatter(c.galactic.l.deg[tau_values>0],c.galactic.b.deg[tau_values>0],c=tau_values[tau_values>0])

In [ ]:
pdf.cre

In [ ]:
pdf.loc[pdf["i:objectId"] == "ZTF25abcwzci", ["d:mulens", "i:objectId", "i:magpsf"]]

In [ ]:
tau_values

In [ ]:
np.tau_values